In [11]:
# Numerical computation & data processing
import os
import copy
import warnings
from datetime import datetime
import gc
import pickle
import joblib

import numpy as np
import pandas as pd

# Machine learning & deep learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.pipeline import Pipeline

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

import optuna

# Statistics & finance tools
import statsmodels.api as sm
from scipy.stats import f as f_dist
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from typing import Optional

optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# Set random seed
torch.manual_seed(42)
np.random.seed(42)

# Device configuration - prefer MPS (Mac GPU), then CUDA, then CPU
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print(f"Using device: {device} (Apple Metal GPU)")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using device: {device} (NVIDIA GPU)")
else:
    device = torch.device('cpu')
    print(f"Using device: {device} (CPU only)")

# MPS specific settings
if device.type == 'mps':
    torch.mps.empty_cache()
    print("MPS cache cleared")

def get_device_info():
    """Get device information"""
    if device.type == 'mps':
        return f"Apple Metal GPU (MPS) - Mac M-series chip"
    elif device.type == 'cuda':
        return f"NVIDIA GPU: {torch.cuda.get_device_name()}"
    else:
        return "CPU"

Using device: mps (Apple Metal GPU)
 MPS cache cleared


In [12]:

def load_y_scalers():
    """Load y scalers for all windows"""
    scalers = {}
    windows = [5, 21, 252, 512]
    for window in windows:
        scaler_path = f"/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/scaler_y_window_{window}.pkl"
        try:
            scalers[window] = joblib.load(scaler_path)
            print(f"[Loaded] Y scaler for window {window}")
        except FileNotFoundError:
            print(f"[Warning] Y scaler not found for window {window}: {scaler_path}")
            scalers[window] = None
        except Exception as e:
            print(f"[Error] Failed to load Y scaler for window {window}: {e}")
            scalers[window] = None
    return scalers

def inverse_transform_y(y_scaled, scaler):
    """Inverse transform standardized y values"""
    if scaler is None:
        print("[Warning] Scaler is None, returning original values")
        return y_scaled
    
    y_scaled = np.array(y_scaled)
    if y_scaled.ndim == 1:
        y_scaled = y_scaled.reshape(-1, 1)
    
    try:
        y_original = scaler.inverse_transform(y_scaled)
        if y_original.shape[1] == 1:
            return y_original.flatten()
        return y_original
    except Exception as e:
        print(f"[Error] Failed to inverse transform: {e}")
        return y_scaled.flatten() if y_scaled.ndim > 1 else y_scaled

Y_SCALERS = load_y_scalers()

def r2_zero(y_true, y_pred):
    """
    Compute zero-based R² (baseline is zero)
    y_true: array of true values (N,)
    y_pred: array of predicted values (N,)
    """
    rss = np.sum((y_true - y_pred)**2)  
    tss = np.sum(y_true**2)            
    return 1 - rss / tss

def calc_ic_daily(df, method='spearman'):
    """
    Calculate daily cross-sectional RankIC.
    df: must contain ['signal_date','y_true','y_pred']
    """
    ics = (df.groupby('signal_date')
             .apply(lambda g: g['y_pred'].corr(g['y_true'], method=method))
             .dropna())
    mean_ic = ics.mean()
    std_ic  = ics.std(ddof=1)
    t_ic    = mean_ic / (std_ic / np.sqrt(len(ics))) if std_ic > 0 else np.nan
    pos_ratio = (ics > 0).mean()
    return mean_ic, t_ic, pos_ratio, ics

def annual_sharpe(rets, freq=252):
    mu = float(np.mean(rets)) * freq
    sd = float(np.std(rets, ddof=1)) * np.sqrt(freq)
    return mu / sd if sd > 0 else 0

def delta_sharpe(r2_zero: float, sr_base: float):
    """
    If r2_zero <= 0   → ΔSharpe = 0, Sharpe* = sr_base
    If r2_zero >= 1   → ΔSharpe = 0, Sharpe* = sr_base (edge case)
    Otherwise, use the original formula
    """
    if (r2_zero <= 0) or (r2_zero >= 1):
        return 0.0, sr_base
    sr_star = np.sqrt(sr_base ** 2 + r2_zero) / np.sqrt(1 - r2_zero)
    return sr_star - sr_base, sr_star

rf_file = "/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/CRSP_2016_2024_top50_with_exret.csv"
rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
rf_df["date"] = pd.to_datetime(rf_df["date"])
rf_df = rf_df.drop_duplicates("date").set_index("date").sort_index()
rf_series = rf_df["rf"].astype(float)

px = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")["Close"]
sp_ret = px.pct_change().dropna()
rf_align = rf_series.reindex(sp_ret.index).fillna(method="ffill")
sp_excess = sp_ret.values - rf_align.values

SR_MKT_EX = annual_sharpe(sp_excess)
print(f"[INFO] S&P500 Excess Sharpe (2016–24) = {SR_MKT_EX:.3f}")

def calc_directional_metrics(y_true, y_pred, permnos=None):

    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    if permnos is None:
        s_true = np.sign(y_true)
        s_pred = np.sign(y_pred)
        mask = s_true != 0
        s_true = s_true[mask]
        s_pred = s_pred[mask]

        overall_acc = np.mean(s_true == s_pred)

        up_mask = s_true > 0
        down_mask = s_true < 0
        up_acc = np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else 0
        down_acc = np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else 0

    else:
        df = pd.DataFrame({"permno": permnos, "yt": y_true, "yp": y_pred})
        overall_accs = []
        up_accs = []
        down_accs = []

        for _, g in df.groupby("permno"):
            s_true = np.sign(g["yt"].values)
            s_pred = np.sign(g["yp"].values)
            mask = s_true != 0
            s_true = s_true[mask]
            s_pred = s_pred[mask]
            if len(s_true) == 0:
                continue
            overall_accs.append(np.mean(s_true == s_pred))

            up_mask = s_true > 0
            down_mask = s_true < 0
            up_accs.append(np.mean(s_true[up_mask] == s_pred[up_mask]) if np.any(up_mask) else np.nan)
            down_accs.append(np.mean(s_true[down_mask] == s_pred[down_mask]) if np.any(down_mask) else np.nan)

        overall_acc = np.nanmean(overall_accs)
        up_acc = np.nanmean(up_accs)
        down_acc = np.nanmean(down_accs)

    return overall_acc, up_acc, down_acc

def regression_metrics(y_true, y_pred, k, meta=None, permnos=None):
    """
    Includes:
    - Regression metrics
    - Pointwise directional accuracy
    - Market cap group metrics
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    n = len(y_true)

    r2 = r2_zero(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    dir_acc, up_acc, down_acc = calc_directional_metrics(y_true, y_pred, permnos)

    metrics = {
        "R²_zero": r2,
        "RMSE": rmse,
        "MAE": mae,
        "MSE": mse,
        "Directional Accuracy": dir_acc,
        "Up_Directional_Acc": up_acc,
        "Down_Directional_Acc": down_acc
    }

    if meta is not None and "MKTCAP_PERCENTILE" in meta:
        top_mask = meta["MKTCAP_PERCENTILE"] >= 0.75
        bottom_mask = meta["MKTCAP_PERCENTILE"] <= 0.25

        if np.any(top_mask):
            yt_top = y_true[top_mask]
            yp_top = y_pred[top_mask]
            perm_top = permnos[top_mask] if permnos is not None else None
            r2_top = r2_zero(yt_top, yp_top)
            rmse_top = np.sqrt(mean_squared_error(yt_top, yp_top))
            mae_top = mean_absolute_error(yt_top, yp_top)
            mse_top = mean_squared_error(yt_top, yp_top)
            dir_top, up_top, down_top = calc_directional_metrics(yt_top, yp_top, perm_top)
            metrics.update({
                "Top25_R2_zero": r2_top,
                "Top25_RMSE": rmse_top,
                "Top25_MAE": mae_top,
                "Top25_MSE": mse_top,
                "Top25_Dir_Acc": dir_top,
                "Top25_Up_Acc": up_top,
                "Top25_Down_Acc": down_top
            })

        if np.any(bottom_mask):
            yt_bot = y_true[bottom_mask]
            yp_bot = y_pred[bottom_mask]
            perm_bot = permnos[bottom_mask] if permnos is not None else None
            r2_bot = r2_zero(yt_bot, yp_bot)
            rmse_bot = np.sqrt(mean_squared_error(yt_bot, yp_bot))
            mae_bot = mean_absolute_error(yt_bot, yp_bot)
            mse_bot = mean_squared_error(yt_bot, yp_bot)
            dir_bot, up_bot, down_bot = calc_directional_metrics(yt_bot, yp_bot, perm_bot)
            metrics.update({
                "Bottom25_R2_zero": r2_bot,
                "Bottom25_RMSE": rmse_bot,
                "Bottom25_MAE": mae_bot,
                "Bottom25_MSE": mse_bot,
                "Bottom25_Dir_Acc": dir_bot,
                "Bottom25_Up_Acc": up_bot,
                "Bottom25_Down_Acc": down_bot
            })

    return metrics

def f_statistic(y_true, y_pred, k):
    """Return F statistic and corresponding p-value"""
    n   = len(y_true)
    rss = np.sum((y_true - y_pred) ** 2)
    tss = np.sum(y_true ** 2)
    r2  = 1 - rss / tss
    if (r2 <= 0) or (n <= k):
        return 0.0, 1.0
    F = (r2 / k) / ((1 - r2) / (n - k))
    p = f_dist.sf(F, k, n - k)
    return F, p

def overall_interval_metrics_method1(y_all, yhat_all, k, permnos_all=None, meta_all=None):
    """
    Method 1: Compute metrics for the entire interval (2016-2024, all samples concatenated)
    Returns: a dict, can be directly passed to save_metrics()
    """
    base = regression_metrics(
        y_true=y_all, 
        y_pred=yhat_all, 
        k=k, 
        meta=meta_all, 
        permnos=permnos_all
    )
    F, p = f_statistic(y_all, yhat_all, k)
    base["F_stat"]     = F
    base["F_pvalue"]   = p
    base["N_obs"] = len(y_all)
    
    delta_cash, sr_star_cash = delta_sharpe(base["R²_zero"], sr_base=0)
    base["ΔSharpe_cash"]      = delta_cash
    base["Sharpe*_cash"]      = sr_star_cash

    delta_mkt , sr_star_mkt  = delta_sharpe(base["R²_zero"], sr_base=SR_MKT_EX)
    base["ΔSharpe_mkt"]       = delta_mkt
    base["Sharpe*_mkt"]       = sr_star_mkt
    
    return base

def sortino_ratio(rets, freq=252):
    """Compute Sortino Ratio"""
    downside = rets[rets < 0]
    if len(downside) == 0:
        return np.inf
    mu = rets.mean() * freq
    sigma = np.sqrt((downside ** 2).mean()) * np.sqrt(freq)
    return mu / sigma

def cvar(rets, alpha=0.95):
    """Compute CVaR"""
    q = np.quantile(rets, 1 - alpha)
    return rets[rets <= q].mean()

def save_predictions(model_name, window_size, y_true, y_pred, permnos, path="predictions/"):
    os.makedirs(path, exist_ok=True)
    
    df = pd.DataFrame({
        "PERMNO": permnos,
        "y_true": y_true,
        "y_pred": y_pred
    })

    filename = f"{model_name}_w{window_size}.csv"
    df.to_csv(os.path.join(path, filename), index=False)
    print(f"[Save] {filename}")

def save_metrics(metrics_dict, name, window, path="results.csv"):
    """Save evaluation metrics"""
    row = pd.DataFrame([metrics_dict])
    row.insert(0, "Model", name)
    row.insert(1, "Window", window)

    if os.path.exists(path):
        df = pd.read_csv(path)
        df = df[~((df["Model"] == name) & (df["Window"] == window))]
        df = pd.concat([df, row], ignore_index=True)
        df.to_csv(path, index=False)
        print(f"[Update] Metrics updated for {name} w={window}")
    else:
        row.to_csv(path, index=False)
        print(f"[Create] New metrics file created with {name} w={window}")

def get_quarter_periods(start_year=2015, end_year=2024):
    """Generate quarter sequence"""
    quarters = []
    for year in range(start_year, end_year + 1):
        for q in range(1, 5):
            quarters.append((year, q))
    return quarters

def save_model_with_quarter(lstm_wrapper, name, window, year, quarter,
                            path="models/"):
    """
    Safely save LSTM model parameters, handle MPS device compatibility and file size optimization
    """
    os.makedirs(path, exist_ok=True)
    
    lstm_wrapper.clear_loss_history()
    
    original_device = lstm_wrapper.training_device
    lstm_wrapper.model.to('cpu')
    
    pth_file = f"{name}_w{window}_{year}Q{quarter}.pth"
    torch.save(lstm_wrapper.model.state_dict(),
               os.path.join(path, pth_file))
    
    lstm_wrapper.model.to(original_device)
    
    print(f"[Saved] {pth_file}")
    
def load_datasets(npz_path):
    """Load dataset"""
    data = np.load(npz_path, allow_pickle=True) 
    datasets = {}
    for key in data.files:
        datasets[key] = data[key]
    return datasets

def find_coef_step(model):
    """
    Get model coefficients, handle Pipeline and single estimator.
    For nonlinear models (e.g., MLP), return None to indicate no coefficients.
    """
    if hasattr(model, 'named_steps'):
        for name, est in model.named_steps.items():
            if hasattr(est, 'coef_'):
                return name, est
            if isinstance(est, Pipeline):
                for subname, subest in est.named_steps.items():
                    if hasattr(subest, 'coef_'):
                        return f"{name}__{subname}", subest
        return None, None
    else:
        if hasattr(model, 'coef_'):
            return 'model', model
    
    raise ValueError("No estimator with coef_ found in model")
    
def train_or_skip(model, train_loader, valid_loader, window_size, year, quarter, **train_kwargs):
    save_path = f"models/NN1_w{window_size}_{year}Q{quarter}.pth"

    if os.path.exists(save_path):
        print(f"[Skip Training] Model already exists: {save_path}")
        return

    print(f"[Training] Start training model: {save_path}")
    train_model(model, train_loader, valid_loader, **train_kwargs)
    torch.save(model.state_dict(), save_path)
    print(f"[Done] Model saved: {save_path}")

[*********************100%***********************]  1 of 1 completed

[Loaded] Y scaler for window 5
[Loaded] Y scaler for window 21
[Loaded] Y scaler for window 252
[Loaded] Y scaler for window 512
[INFO] S&P500 Excess Sharpe (2016–24) = 0.652


In [13]:
# ------------------------------------------------------------------
# 0) Utilities (keep the same interface as the original script)
# ------------------------------------------------------------------

def get_default_device() -> torch.device:
    """Automatically select GPU / MPS / CPU"""
    if torch.cuda.is_available():
        return torch.device("cuda")
    if torch.backends.mps.is_available():
        return torch.device("mps")
    return torch.device("cpu")
    
DEVICE_TUNE  = torch.device("cpu")
DEVICE_TRAIN = get_default_device()

def clear_memory():
    gc.collect()
    """Release GPU / MPS memory, no-op for CPU"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

# ------------------------------------------------------------------
# 1) Early Stopping
# ------------------------------------------------------------------

class EarlyStopping:
    """Same as original API, just set default patience to 10"""

    def __init__(self, patience: int = 10, min_delta: float = 0.0, restore_best_weights: bool = True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss: Optional[float] = None
        self.counter = 0
        self.best_weights = None

    def __call__(self, val_loss: float, model: nn.Module) -> bool:
        if self.best_loss is None:
            self.best_loss = val_loss
            self._save(model)
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self._save(model)
        else:
            self.counter += 1

        if self.counter >= self.patience:
            if self.restore_best_weights and self.best_weights is not None:
                model.load_state_dict(self.best_weights)
            return True
        return False

    def _save(self, model: nn.Module):
        # Move weights to CPU, replace original copy.deepcopy
        self.best_weights = {
            k: v.detach().to("cpu")
            for k, v in model.state_dict().items()
        }

# ------------------------------------------------------------------
# 2) LSTM Base Model
# ------------------------------------------------------------------

class LSTMModel(nn.Module):
    def __init__(
        self,
        input_size: int,
        seq_len: int,
        hidden_size: int = 64,
        num_layers: int = 2,
        dropout: float = 0.2,
    ) -> None:
        super().__init__()
        self.seq_len = seq_len
        self.features_per_ts = input_size // seq_len
        assert (
            input_size % seq_len == 0
        ), "input_size must be divisible by seq_len for reshape -> (B, seq_len, feat_per_ts)"

        self.lstm = nn.LSTM(
            input_size=self.features_per_ts,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0.0,
            batch_first=True,
        )
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_size, 1)

        self.apply(self._init_weights)

    @staticmethod
    def _init_weights(m):
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LSTM):
            for name, p in m.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(p)
                else:
                    nn.init.constant_(p, 0)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (B, input_size)  OR (B, seq_len, feat_per_ts)
        if x.ndim == 2:
            bsz = x.size(0)
            x = x.view(bsz, self.seq_len, self.features_per_ts)
        lstm_out, _ = self.lstm(x)
        last = lstm_out[:, -1, :]  # (B, hidden)
        last = self.dropout(last)
        return self.out(last).squeeze(-1)

# ------------------------------------------------------------------
# 3) LSTM Wrapper 
# ------------------------------------------------------------------

class LSTMWrapper:
   

    def __init__(
        self,
        input_size: int,
        seq_len: int,
        hidden_size: int = 64,
        num_layers: int = 2,
        dropout: float = 0.2,
        learning_rate: float = 1e-3,
        batch_size: int = 256,
        max_epochs: int = 40,
        warm_start_epochs: int = 10,
        training_device: Optional[torch.device] = None,
    ):
        self.input_size = input_size
        self.seq_len = seq_len
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.max_epochs = max_epochs
        self.warm_start_epochs = warm_start_epochs
        self.training_device = training_device or get_default_device()

        self._build_model()
        self.is_fitted = False
        self.loss_history: dict[str, list[float]] = {}

    def _build_model(self):
        self.model = LSTMModel(
            input_size=self.input_size,
            seq_len=self.seq_len,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            dropout=self.dropout,
        ).to(self.training_device)
        self._init_training_components()

    def _init_training_components(self):
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.scheduler = ReduceLROnPlateau(self.optimizer, mode="min", factor=0.5, patience=5, verbose=False)
        self.criterion = nn.MSELoss()
        self.early_stopping = EarlyStopping(patience=5)

    def _make_loaders(self, X, y, validation_split):
        val_size = int(len(X) * validation_split) if validation_split > 0 else max(1, int(len(X) * 0.1))
        X_train, X_val = X[:-val_size], X[-val_size:]
        y_train, y_val = y[:-val_size], y[-val_size:]

        train_ds = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
        val_ds = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
        train_loader = DataLoader(train_ds, batch_size=self.batch_size, shuffle=False, drop_last=True)
        val_loader = DataLoader(val_ds, batch_size=self.batch_size, shuffle=False)
        return train_loader, val_loader

    def fit(self, X, y, validation_split: float = 0.1, warm_start: bool = False, verbose: bool = True):
        if not warm_start:
            self._build_model()
        else:
            if verbose:
                print("    [Warm Start] continue training existing LSTM weights ...")
            self.early_stopping = EarlyStopping(patience=5)

        train_loader, val_loader = self._make_loaders(X, y, validation_split)
        epochs = self.warm_start_epochs if warm_start else self.max_epochs
        if verbose:
            print(f"    Training LSTM for {epochs} epochs on {self.training_device}")

        losses = {"train": [], "val": []}
        for ep in range(epochs):
            self.model.train()
            train_loss = 0.0
            for bx, by in train_loader:
                bx, by = bx.to(self.training_device), by.to(self.training_device)
                self.optimizer.zero_grad()
                preds = self.model(bx)
                loss = self.criterion(preds, by)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()
            train_loss /= len(train_loader)

            self.model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for bx, by in val_loader:
                    bx, by = bx.to(self.training_device), by.to(self.training_device)
                    loss = self.criterion(self.model(bx), by)
                    val_loss += loss.item()
            val_loss /= len(val_loader)

            losses["train"].append(train_loss)
            losses["val"].append(val_loss)

            if verbose and ((ep + 1) % 5 == 0 or ep == 0):
                print(f"    Epoch {ep+1:3d}/{epochs}: Train={train_loss:.6f}, Val={val_loss:.6f}")

            self.scheduler.step(val_loss)
            if self.early_stopping(val_loss, self.model):
                if verbose:
                    print(f"    Early-stopped at epoch {ep+1}")
                break

            if self.training_device.type == "mps" and (ep + 1) % 5 == 0:
                torch.mps.empty_cache()
                gc.collect()

        self.loss_history = losses
        clear_memory()
        self.is_fitted = True

    def predict(self, X):
        if not self.is_fitted:
            raise RuntimeError("Call fit() before predict()")
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X).to(self.training_device)
            preds = self.model(X_tensor).cpu().numpy()
        return preds

    def partial_fit(self, X_new, y_new, validation_split: float = 0.1, extra_epochs: int = 5, verbose: bool = True):
        if not self.is_fitted:
            raise RuntimeError("Must fit() once before partial_fit()")
        self.early_stopping = EarlyStopping(patience=5)
        if verbose:
            print(f"    Partial-fit on {len(X_new)} samples for {extra_epochs} epochs ...")

        train_loader, val_loader = self._make_loaders(X_new, y_new, validation_split)
        for ep in range(extra_epochs):
            self.model.train()
            train_loss = 0.0
            for bx, by in train_loader:
                bx, by = bx.to(self.training_device), by.to(self.training_device)
                self.optimizer.zero_grad()
                loss = self.criterion(self.model(bx), by)
                loss.backward()
                self.optimizer.step()
                train_loss += loss.item()
            train_loss /= len(train_loader)

            self.model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for bx, by in val_loader:
                    bx, by = bx.to(self.training_device), by.to(self.training_device)
                    val_loss += self.criterion(self.model(bx), by).item()
            val_loss /= len(val_loader)

            if verbose and (ep % 2 == 0 or ep == extra_epochs - 1):
                print(f"        Epoch {ep+1:2d}/{extra_epochs}: Train={train_loss:.6f}, Val={val_loss:.6f}")

            self.scheduler.step(val_loss)
            if self.training_device.type == "mps" and (ep + 1) % 5 == 0:
                torch.mps.empty_cache()
                gc.collect()
            if self.early_stopping(val_loss, self.model):
                if verbose:
                    print(f"        Early-stopped at epoch {ep+1}")
                break
        clear_memory()

    def clear_loss_history(self):
        if hasattr(self, "loss_history"):
            del self.loss_history

    def set_params(self, **params):
        for k, v in params.items():
            if hasattr(self, k):
                setattr(self, k, v)
        self._init_training_components()

In [14]:
# ========== 1. Only window=5 uses Optuna tuning ===============================
TUNED_WINDOW = 5  # Other windows use the best hyperparameters from window=5

def tune_lstm_with_optuna(X, y, seq_len, n_trials: int = 10):
    """LSTM hyperparameter search with time series CV (only called for window=5)"""

    tscv = TimeSeriesSplit(n_splits=3)
    total_feat = X.shape[1]
    print("Start hyperparameter tuning...")
    def objective(trial):
        params = dict(
            batch_size   = trial.suggest_categorical("batch_size",  [32, 64, 128]),
            learning_rate= trial.suggest_float("learning_rate",   1e-4, 1e-2, log=True),
            dropout      = trial.suggest_float("dropout",         0.0,  0.3),
            hidden_size  = trial.suggest_categorical("hidden_size", [64, 128]),
            num_layers   = trial.suggest_int("num_layers",        1, 3),
            max_epochs   = 10,          # Use short epochs for tuning
        )

        cv_mse = []
        for tr_idx, val_idx in tscv.split(X):
            X_tr, X_val = X[tr_idx], X[val_idx]
            y_tr, y_val = y[tr_idx], y[val_idx]

            model = LSTMWrapper(
                input_size   = total_feat,
                seq_len      = seq_len,
                hidden_size  = params["hidden_size"],
                num_layers   = params["num_layers"],
                dropout      = params["dropout"],
                learning_rate= params["learning_rate"],
                batch_size   = params["batch_size"],
                max_epochs   = params["max_epochs"],
                training_device  = DEVICE_TUNE,
            )
            model.fit(X_tr, y_tr, validation_split=0.0, warm_start=False)
            preds = model.predict(X_val)
            cv_mse.append(mean_squared_error(y_val, preds))
            del model
            clear_memory()
        return float(np.mean(cv_mse))

    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, n_jobs=1)

    if study.best_trial is None:
        # Default fallback
        return dict(batch_size=32, learning_rate=1e-3, dropout=0.2,
                    hidden_size=64, num_layers=2, max_epochs=25)

    best = study.best_params
    best["max_epochs"] = 25          # Use more epochs for final training
    print(f"[Optuna-LSTM] best_MSE={study.best_value:.6f}, params={best}")
    return best

# ========== 2. Model saving / loading ============================

def _lstm_save_path(name: str, window: int, year: int, quarter: int | None):
    if quarter is None:
        return f"models/{name}_w{window}_{year}.pth"
    return f"models/{name}_w{window}_{year}Q{quarter}.pth"


def save_lstm_model(wrapper: LSTMWrapper,
                    name: str, window: int, year: int, quarter: int | None = None):
    """
    Save model weights and structure hyperparameters
    """
    os.makedirs("models", exist_ok=True)
    path = _lstm_save_path(name, window, year, quarter)

    ckpt = {
        "state_dict": wrapper.model.state_dict(),
        "hyper_params": dict(
            input_size   = wrapper.input_size,
            seq_len      = wrapper.seq_len,
            hidden_size  = wrapper.hidden_size,
            num_layers   = wrapper.num_layers,
            dropout      = wrapper.dropout,
            learning_rate= wrapper.learning_rate,
            batch_size   = wrapper.batch_size,
            max_epochs   = wrapper.max_epochs,
        )
    }

    torch.save(ckpt, path)
    print(f"[Saved] {path}")


def load_lstm_model(name: str, window: int, year: int,
                    quarter: int | None = None,
                    training_device: torch.device | None = None):
    """
    Load checkpoint from disk and restore LSTMWrapper (including structure hyperparameters)
    """
    path = _lstm_save_path(name, window, year, quarter)
    if not os.path.exists(path):
        return None                       # Return None if file does not exist

    ckpt = torch.load(path, map_location="cpu")

    hp = ckpt["hyper_params"]
    wrapper = LSTMWrapper(
        input_size     = hp["input_size"],
        seq_len        = hp["seq_len"],
        hidden_size    = hp["hidden_size"],
        num_layers     = hp["num_layers"],
        dropout        = hp["dropout"],
        learning_rate  = hp.get("learning_rate", 1e-3),
        batch_size     = hp.get("batch_size", 32),
        max_epochs     = hp.get("max_epochs", 25),
        training_device= training_device or DEVICE_TRAIN,
    )
    wrapper.model.load_state_dict(ckpt["state_dict"])
    wrapper.is_fitted = True
    return wrapper

# ========== 3. Quarterly expanding training ================================

def train_lstm_models_expanding_quarterly(start_year: int = 2015, end_year: int = 2024,
                                          window_sizes: list[int] | None = None,
                                          npz_path: str = "/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/all_window_datasets_scaled.npz",
                                          n_trials_optuna: int = 10):
    
    if window_sizes is None:
        window_sizes = [5, 21, 252, 512]

    print(f"Starting LSTM quarterly expanding training {start_year}-{end_year}")
    data = load_datasets(npz_path)

    best_params_cache: dict[str, dict] = {}
    quarters_to_tune = {(2020, 4)}

    for window in window_sizes:
        print(f"\n=== Window = {window} ===")
        X_train_init = data[f"X_train_{window}"]
        y_train_init = data[f"y_train_{window}"]
        X_test_full  = data[f"X_test_{window}"]
        y_test_full  = data[f"y_test_{window}"]
        meta_test    = pd.DataFrame.from_dict(data[f"meta_test_{window}"].item())
        meta_test["ret_date"] = pd.to_datetime(meta_test["ret_date"])

        total_feat = X_train_init.shape[1]
        feat_per_ts = total_feat

        # Initial tuning (only window=5 / 2015Q4)
        cache_key = f"LSTM_w{window}"
        if window == TUNED_WINDOW:
            tuned_model = load_lstm_model("LSTM", window, 2015, 4)
            if tuned_model is not None:
                hp = dict(
                    hidden_size   = tuned_model.hidden_size,
                    num_layers    = tuned_model.num_layers,
                    dropout       = tuned_model.dropout,
                    learning_rate = tuned_model.learning_rate,
                    batch_size    = tuned_model.batch_size,
                    max_epochs    = tuned_model.max_epochs,
                )
                print("[Skip-Optuna] hyper-params loaded from existing 2015Q4 model")
            else:
                print("  - Optuna tuning on initial window...")
                hp = tune_lstm_with_optuna(X_train_init, y_train_init, window, n_trials_optuna)
        else:
            hp = None
        best_params_cache[cache_key] = hp
        
        model_prev = None
        # Quarter loop
        for year, quarter in get_quarter_periods(start_year, end_year):
            model_cur = load_lstm_model("LSTM", window, year, quarter)
            if model_cur is not None:
                print(f"[Skip] Model already trained for window={window}, {year}Q{quarter}")
                continue
            if (year == start_year and quarter < 4) or (year == end_year and quarter > 3):
                continue  

            print(f"\n[Window {window}] {year}Q{quarter}")

            # Prepare expanding dataset
            if not (year == start_year and quarter == 4):
                if quarter == 1:
                    py, pq = year - 1, 4
                else:
                    py, pq = year, quarter - 1
                mask_prev = (meta_test["ret_date"].dt.year == py) & (meta_test["ret_date"].dt.quarter == pq)
                if mask_prev.any():
                    X_prev, y_prev = X_test_full[mask_prev], y_test_full[mask_prev]
                    X_train_init = np.vstack([X_train_init, X_prev])
                    y_train_init = np.hstack([y_train_init, y_prev])
                    print(f"    +{mask_prev.sum()} obs from {py}Q{pq} -> expanding size {len(y_train_init)}")

            # Hyperparameter copy / re-tune
            cache_key = f"LSTM_w{window}"
            hp = best_params_cache.get(cache_key)
            if hp is None and window != TUNED_WINDOW:
                hp = best_params_cache[f"LSTM_w{TUNED_WINDOW}"].copy()
                best_params_cache[cache_key] = hp

            if (year, quarter) in quarters_to_tune and window == TUNED_WINDOW:
                print("    Re-tuning via Optuna...")
                hp = tune_lstm_with_optuna(X_train_init, y_train_init, window, n_trials_optuna)
                best_params_cache[cache_key] = hp

            # Load previous model for warm-start
            model_prev = None
            if not (year == start_year and quarter == 4):
                if quarter == 1:
                    prev_year, prev_quarter = year - 1, 4
                else:
                    prev_year, prev_quarter = year, quarter - 1

                model_prev = load_lstm_model("LSTM", window, prev_year, prev_quarter)

            if model_prev is not None:
                print("    Warm-start from last quarter model ...")
                model_prev.partial_fit(X_train_init, y_train_init, validation_split=0.1, extra_epochs=hp.get("warm_start_epochs", 10))
                lstm_wrap = model_prev
            else:
                print("    Cold start training ...")
                lstm_wrap = LSTMWrapper(
                    input_size   = total_feat,
                    seq_len      = window,
                    hidden_size  = hp["hidden_size"],
                    num_layers   = hp["num_layers"],
                    dropout      = hp["dropout"],
                    learning_rate= hp["learning_rate"],
                    batch_size   = hp["batch_size"],
                    max_epochs   = hp["max_epochs"],
                    training_device       = DEVICE_TRAIN,
                )
                lstm_wrap.fit(X_train_init, y_train_init, validation_split=0.1, warm_start=False)

            save_lstm_model(lstm_wrap, "LSTM", window, year, quarter)
            del lstm_wrap
            clear_memory()
            
    print("All LSTM quarterly expanding models trained")


In [15]:
# ===== Portfolio Core Class =====
# ===== Transaction Cost Settings =====
TC_GRID = [0.0005, 0.001, 0.002, 0.003, 0.004]  # 5, 10, 20, 30, 40 bps
TC_TAG  = {
    0.0005: "tc5",
    0.001:  "tc10", 
    0.002:  "tc20",
    0.003:  "tc30",
    0.004:  "tc40"
}

class PortfolioBacktester:
    def __init__(self):
        self.results = {}
        
    def calc_turnover(self, w_t, r_t, w_tp1):
        
        if w_t is None:
            return np.sum(np.abs(w_tp1))
        
        gross_ret = np.sum(w_t * r_t)
        if abs(1 + gross_ret) < 1e-8:  # Avoid division by zero
            return np.sum(np.abs(w_tp1))
        
        passive_weight = w_t * (1 + r_t) / (1 + gross_ret)
        turnover = np.sum(np.abs(w_tp1 - passive_weight))
        return turnover
    
    def create_portfolios_with_permno_tracking(self, signals, market_caps, permnos, top_pct=0.1, bottom_pct=0.1, weight_scheme="VW"):
        """
        Create portfolio weights based on signals, strictly tracking permno alignment.
        weight_scheme: 'VW' for value-weighted, 'EW' for equal-weighted
        """
        n_stocks = len(signals)
        top_n    = max(1, int(round(n_stocks * top_pct)))
        bottom_n = max(1, int(round(n_stocks * bottom_pct)))
        
        sorted_idx = np.argsort(signals)[::-1]
        
        top_idx = sorted_idx[:top_n]
        bottom_idx = sorted_idx[-bottom_n:]
        
        portfolio_data = {}
        
        long_weights = np.zeros(n_stocks)
        if len(top_idx) > 0:
            if weight_scheme == "VW":
                top_market_caps = market_caps[top_idx]
                if np.sum(top_market_caps) > 0:
                    long_weights[top_idx] = top_market_caps / np.sum(top_market_caps)
            else:
                long_weights[top_idx] = 1.0 / len(top_idx)
        
        portfolio_data['long_only'] = {
            'weights': long_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[top_idx] if len(top_idx) > 0 else np.array([])
        }
        
        short_weights = np.zeros(n_stocks)
        if len(bottom_idx) > 0:
            if weight_scheme == "VW":
                bottom_market_caps = market_caps[bottom_idx]
                if np.sum(bottom_market_caps) > 0:
                    short_weights[bottom_idx] = -bottom_market_caps / np.sum(bottom_market_caps)
            else:
                short_weights[bottom_idx] = -1.0 / len(bottom_idx)
        
        portfolio_data['short_only'] = {
            'weights': short_weights,
            'permnos': permnos.copy(),
            'selected_permnos': permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        }
        
        # Long-Short portfolio (Top long + Bottom short)
        ls_raw = long_weights + short_weights

        gross_target = 2.0
        current_gross = np.sum(np.abs(long_weights)) + np.sum(np.abs(short_weights))
        scale = gross_target / current_gross if current_gross > 1e-8 else 0.0
        ls_weights = scale * ls_raw

        ls_selected_permnos = np.concatenate([
            permnos[top_idx] if len(top_idx) > 0 else np.array([]),
            permnos[bottom_idx] if len(bottom_idx) > 0 else np.array([])
        ])

        portfolio_data['long_short'] = {
            'weights': ls_weights,
            'permnos': permnos.copy(),
            'selected_permnos': ls_selected_permnos
        }

        return portfolio_data
    
    def calculate_aligned_portfolio_return(self, portfolio_weights, portfolio_permnos, actual_returns, actual_permnos):
        """Calculate portfolio return strictly aligned by permno"""
        aligned_returns = np.zeros(len(portfolio_permnos))
        
        return_dict = dict(zip(actual_permnos, actual_returns))
        
        for i, permno in enumerate(portfolio_permnos):
            if permno in return_dict:
                aligned_returns[i] = return_dict[permno]
        
        portfolio_return = np.sum(portfolio_weights * aligned_returns)
        return portfolio_return, aligned_returns

    def calculate_metrics(self, returns, turnover_series=None):
        """Calculate portfolio metrics - only returns summary metrics, not long series"""
        returns = np.array(returns)
        
        annual_return = np.mean(returns) * 252
        annual_vol = np.std(returns, ddof=1) * np.sqrt(252)
        sharpe = annual_return / annual_vol if annual_vol > 0 else 0
        
        log_cum = np.cumsum(np.log1p(returns))
        peak_log = np.maximum.accumulate(log_cum)
        dd_log = peak_log - log_cum
        max_drawdown = 1 - np.exp(-dd_log.max()) 
        max_1d_loss = np.min(returns) 
        
        avg_turnover = np.mean(turnover_series) if turnover_series is not None else 0
        
        sortino = sortino_ratio(returns)
        cvar95  = cvar(returns, alpha=0.95)

        result = {
            'annual_return': annual_return,
            'annual_vol': annual_vol,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'max_1d_loss': max_1d_loss,
            'avg_turnover': avg_turnover,
            'sortino': sortino,
            'cvar95': cvar95
        }
        
        return result

In [16]:
# ========== Main function for daily prediction and next-day rebalancing portfolio simulation ==========

def run_portfolio_simulation_daily_rebalance(start_year=2016, end_year=2024, window_sizes=None, model_names=None,
                                           npz_path="/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/all_window_datasets_scaled.npz"):
    """
Portfolio simulation (daily prediction, next-day rebalancing):
    1. Load quarterly models (trained with quarterly expanding window)
    2. Daily prediction to daily signals
    3. Daily portfolio construction (T+1 rebalancing, strict permno alignment)
    4. Separate summary metrics and time series data
    """
    if window_sizes is None:
        window_sizes = [5, 21, 252, 512]
    if model_names is None:
        model_names = ["LSTM"]
    
    print("Starting Daily Rebalance Portfolio Backtesting Simulation")
    
    backtester = PortfolioBacktester()
    datasets = load_datasets(npz_path)
    
    summary_results = []
    daily_series_data = []
    pred_rows = []
    
    WEIGHT_SCHEMES = ["VW", "EW"]
    
    for window in window_sizes:
        print(f"Processing window size: {window}")
        
        X_test = datasets[f"X_test_{window}"]
        y_test = datasets[f"y_test_{window}"]
        input_size = X_test.shape[1]
        meta_test_dict = datasets[f"meta_test_{window}"].item()
        meta_test = pd.DataFrame.from_dict(meta_test_dict)
        
        permnos_test = meta_test["PERMNO"].values
        meta_test["signal_date"]  = pd.to_datetime(meta_test["date"])
        meta_test["ret_date"]     = pd.to_datetime(meta_test["ret_date"])
        market_caps = meta_test.get("MKTCAP", np.ones(len(permnos_test)))
        
        meta_test['date'] = pd.to_datetime(meta_test["date"])
        dates_test = meta_test['signal_date']
        
        for model_name in model_names:
            for scheme in WEIGHT_SCHEMES:
                all_y_true   = []
                all_y_pred   = []
                all_permnos  = []
                all_meta     = []
                print(f"  Model: {model_name}, Scheme: {scheme}")
                
                portfolio_daily_data = {
                    'long_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'short_only': {'returns': [], 'turnovers': [], 'dates': []},
                    'long_short': {'returns': [], 'turnovers': [], 'dates': []}
                }
                
                prev_portfolio_data = {'long_only': None, 'short_only': None, 'long_short': None}
                
                signals_buf = {}
                
                for year in range(start_year, min(end_year + 1, 2025)):
                    for quarter in range(1, 5):
                        # Determine model file year and quarter (T+1 logic: use previous quarter's model to predict current quarter)
                        if quarter == 1:
                            model_file_year, model_file_quarter = year - 1, 4
                        else:
                            model_file_year, model_file_quarter = year, quarter - 1
                            
                        pth_path = f"models/{model_name}_w{window}_{model_file_year}Q{model_file_quarter}.pth"
                        if not os.path.exists(pth_path):
                            print(f"      Skip: Model file not found {pth_path}")
                            continue
                        cpu = torch.device('cpu')
                        ckpt = torch.load(pth_path, map_location=cpu)

                        hp = ckpt.get("hyper_params", {})
                        hidden_size  = hp.get("hidden_size", 64)
                        num_layers   = hp.get("num_layers", 2)
                        dropout      = hp.get("dropout", 0.2)
                        batch_size   = hp.get("batch_size", 256)
                        learning_rate= hp.get("learning_rate", 1e-3)
                        max_epochs   = hp.get("max_epochs", 0)

                        lstm = LSTMWrapper(
                            input_size=input_size,
                            seq_len=window,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            dropout=dropout,
                            learning_rate=learning_rate,
                            batch_size=batch_size,
                            max_epochs=max_epochs,
                            training_device=cpu
                        )

                        if "state_dict" in ckpt:
                            state_dict = ckpt["state_dict"]
                        elif "model_state_dict" in ckpt:
                            state_dict = ckpt["model_state_dict"]
                        else:
                            state_dict = ckpt

                        lstm.model.load_state_dict(state_dict, strict=False)
                        lstm.is_fitted = True
                        model = lstm

                        quarter_mask = (
                            (dates_test.dt.year == year) & 
                            (dates_test.dt.quarter == quarter)
                        )
                        if not np.any(quarter_mask):
                            continue
                        
                        X_quarter = X_test[quarter_mask]
                        y_quarter = y_test[quarter_mask]
                        permnos_quarter = permnos_test[quarter_mask]
                        market_caps_quarter = market_caps[quarter_mask]
                        dates_quarter = dates_test[quarter_mask]
                        ret_dates_quarter = meta_test.loc[quarter_mask, 'ret_date'].values
                        
                        y_pred_scaled = model.predict(X_quarter)
                        
                        y_scaler = Y_SCALERS.get(window)
                        if y_scaler is not None:
                            y_pred_original = inverse_transform_y(y_pred_scaled, y_scaler)
                            y_true_original = inverse_transform_y(y_quarter, y_scaler)
                            print(f"[Inverse Transform] Window {window}: predictions and actuals converted to original scale")
                        else:
                            print(f"[Warning] No scaler found for window {window}, using scaled values")
                            y_pred_original = y_pred_scaled
                            y_true_original = y_quarter
                        
                        df_quarter = pd.DataFrame({
                            'signal_date': dates_quarter,
                            'ret_date': ret_dates_quarter,
                            'permno': permnos_quarter,
                            'market_cap': market_caps_quarter,
                            'actual_return': y_true_original,                 
                            'prediction': y_pred_original   
                        })
                        
                        if scheme == 'VW':
                            df_q_save = df_quarter[['signal_date','ret_date','permno',
                                                    'actual_return','prediction','market_cap']].copy()
                            df_q_save.rename(columns={'actual_return':'y_true',
                                                      'prediction':'y_pred'}, inplace=True)
                            df_q_save['model']  = model_name
                            df_q_save['window'] = window
                            pred_rows.append(df_q_save)
                        
                        all_y_true.append(df_quarter['actual_return'].values)
                        all_y_pred.append(df_quarter['prediction'].values)
                        all_permnos.append(df_quarter['permno'].values)
                        all_meta.append(meta_test.loc[quarter_mask, :])   

                        for signal_date, sig_grp in df_quarter.groupby('signal_date'):
                            daily_signals = (
                                sig_grp.groupby('permno')['prediction'].mean()
                                      .to_frame('prediction')
                                      .join(sig_grp.groupby('permno')['market_cap'].mean())
                            )
                            signals_buf[signal_date] = daily_signals

                            prev_date = signal_date - pd.tseries.offsets.BDay(1)
                            if prev_date not in signals_buf:
                                continue

                            sigs = signals_buf.pop(prev_date)
                            if prev_date in signals_buf:
                                del signals_buf[prev_date]

                            ret_grp = df_quarter[df_quarter['ret_date'] == signal_date]
                            if len(ret_grp) == 0:
                                continue

                            daily_actual_returns = (
                                ret_grp.groupby('permno')['actual_return']
                                       .mean()
                                       .reindex(sigs.index, fill_value=0)
                                       .values
                            )
                            daily_permnos = sigs.index.values

                            portfolios_data = backtester.create_portfolios_with_permno_tracking(
                                signals      = sigs['prediction'].values,
                                market_caps  = sigs['market_cap'].values,
                                permnos      = daily_permnos,
                                weight_scheme= scheme
                            )
                            
                            for portfolio_type in ['long_only', 'short_only', 'long_short']:
                                portfolio_info = portfolios_data[portfolio_type]
                                
                                portfolio_return, aligned_returns = backtester.calculate_aligned_portfolio_return(
                                    portfolio_weights=portfolio_info['weights'],
                                    portfolio_permnos=portfolio_info['permnos'],
                                    actual_returns=daily_actual_returns,
                                    actual_permnos=daily_permnos
                                )
                                
                                if prev_portfolio_data[portfolio_type] is not None:
                                    prev_w_ser = pd.Series(
                                        prev_portfolio_data[portfolio_type]['weights'],
                                        index=prev_portfolio_data[portfolio_type]['permnos']
                                    )
                                    cur_w_ser = pd.Series(
                                        portfolio_info['weights'],
                                        index=portfolio_info['permnos']
                                    )

                                    prev_r_ser = pd.Series(
                                        prev_portfolio_data[portfolio_type]['aligned_returns'],
                                        index=prev_portfolio_data[portfolio_type]['permnos']
                                    )

                                    aligned_prev_w = prev_w_ser.reindex(cur_w_ser.index, fill_value=0).values
                                    aligned_prev_r = prev_r_ser.reindex(cur_w_ser.index, fill_value=0).values

                                    aligned_cur_w = cur_w_ser.values

                                    turnover = backtester.calc_turnover(
                                        w_t  = aligned_prev_w,
                                        r_t  = aligned_prev_r,
                                        w_tp1= aligned_cur_w
                                    )
                                else:
                                    turnover = np.sum(np.abs(portfolio_info['weights']))
                                
                                portfolio_daily_data[portfolio_type]['returns'].append(portfolio_return)
                                portfolio_daily_data[portfolio_type]['turnovers'].append(turnover)
                                portfolio_daily_data[portfolio_type]['dates'].append(signal_date)
                                
                                prev_portfolio_data[portfolio_type] = {
                                    'weights'        : portfolio_info['weights'],
                                    'permnos'        : portfolio_info['permnos'],
                                    'aligned_returns': aligned_returns      
                                }
                
                for portfolio_type in ['long_only', 'short_only', 'long_short']:
                    portfolio_data = portfolio_daily_data[portfolio_type]
                    
                    if len(portfolio_data['returns']) > 0:
                        metrics = backtester.calculate_metrics(
                            returns=portfolio_data['returns'],
                            turnover_series=portfolio_data['turnovers']
                        )
                        
                        rets = np.array(portfolio_data['returns'])
                        tovs = np.array(portfolio_data['turnovers'])

                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            adj = rets - tovs * tc

                            ann_ret = adj.mean() * 252
                            ann_vol = adj.std(ddof=1) * np.sqrt(252)
                            sharpe  = ann_ret / ann_vol if ann_vol > 0 else 0

                            cum_adj = np.cumprod(1 + adj)
                            mdd = ((cum_adj - np.maximum.accumulate(cum_adj)) /
                                   np.maximum.accumulate(cum_adj)).min()

                            metrics[f'{tag}_annual_return'] = ann_ret
                            metrics[f'{tag}_annual_vol']    = ann_vol
                            metrics[f'{tag}_sharpe']        = sharpe
                            metrics[f'{tag}_max_drawdown']  = mdd
                        
                        summary_results.append({
                            'scheme': scheme,
                            'model': model_name,
                            'window': window,
                            'portfolio_type': portfolio_type,
                            **metrics
                        })
                        
                        rets_arr = np.array(portfolio_data['returns'])
                        tovs_arr = np.array(portfolio_data['turnovers'])
                        cum_no_tc = np.log1p(rets_arr).cumsum()

                        tc_ret_dict = {}
                        tc_cum_dict = {}
                        for tc in TC_GRID:
                            tag = TC_TAG[tc]
                            r = rets_arr - tovs_arr * tc
                            tc_ret_dict[tag] = r
                            tc_cum_dict[tag] = np.log1p(r).cumsum()

                        for i, date in enumerate(portfolio_data['dates']):
                            row = {
                                'scheme'        : scheme,
                                'model'         : model_name,
                                'window'        : window,
                                'portfolio_type': portfolio_type,
                                'date'          : str(date),
                                'return'        : rets_arr[i],
                                'turnover'      : tovs_arr[i],
                                'cumulative'    : cum_no_tc[i],
                            }
                            for tag in TC_TAG.values():
                                row[f'{tag}_return']     = tc_ret_dict[tag][i]
                                row[f'{tag}_cumulative'] = tc_cum_dict[tag][i]

                            daily_series_data.append(row)

                if scheme == "VW" and len(all_y_true) > 0:
                    y_all    = np.concatenate(all_y_true)
                    yhat_all = np.concatenate(all_y_pred)
                    perm_all = np.concatenate(all_permnos)
                    meta_all = pd.concat(all_meta, ignore_index=True)

                    k = X_test.shape[1]

                    m1_metrics = overall_interval_metrics_method1(
                        y_all, yhat_all, k,
                        permnos_all=perm_all,
                        meta_all=meta_all
                    )

                    full_pred_df = pd.concat(pred_rows, ignore_index=True)
                    mean_ic, t_ic, pos_ic, _ = calc_ic_daily(full_pred_df, method='spearman')
                    m1_metrics['RankIC_mean']  = mean_ic
                    m1_metrics['RankIC_t']     = t_ic
                    m1_metrics['RankIC_pos%']  = pos_ic

                    save_metrics(m1_metrics, name=model_name, window=window,
                        path="portfolio_metrics.csv")

    summary_df = pd.DataFrame(summary_results)
    daily_df = pd.DataFrame(daily_series_data) if daily_series_data else pd.DataFrame()
    
    tc_columns = [c for c in summary_df.columns if c.startswith('tc')]
    summary_df[tc_columns] = summary_df[tc_columns].fillna(0.0)
    
    def save_split_by_scheme(df, base_filename):
        """Helper function to save files split by scheme"""
        if df.empty:
            print(f"Warning: DataFrame is empty, skipping save for {base_filename}")
            return None, None
            
        vw_df = df[df['scheme'] == 'VW']
        ew_df = df[df['scheme'] == 'EW']
        
        vw_filename = f"{base_filename}_VW.csv"
        ew_filename = f"{base_filename}_EW.csv"
        
        vw_df.to_csv(vw_filename, index=False)
        ew_df.to_csv(ew_filename, index=False)
        
        print(f"VW results saved to {vw_filename}")
        print(f"EW results saved to {ew_filename}")
        
        return vw_filename, ew_filename
    
    save_split_by_scheme(summary_df, "portfolio_results_daily_rebalance")
    
    if not daily_df.empty:
        save_split_by_scheme(daily_df, "portfolio_daily_series")
    
    if pred_rows:
        pred_df = pd.concat(pred_rows, ignore_index=True)
        pred_df.to_csv("predictions_daily.csv", index=False)
        print(f"Saved {len(pred_df)} prediction rows to predictions_daily.csv")
    
    print(f"Generated {len(summary_results)} portfolio summary records")
    print(f"Generated {len(daily_series_data)} daily series records")
    
    return summary_df, daily_df, backtester


### Test

In [18]:
train_lstm_models_expanding_quarterly()

Starting **LSTM** quarterly expanding training 2015‑2024

=== Window = 5 ===
  – Optuna tuning on initial window…
Start hyperparameter tuning...
    Training LSTM for 10 epochs on cpu
    Epoch   1/10: Train=1.890126, Val=0.842051
    Epoch   5/10: Train=1.886850, Val=0.843425
    Early‑stopped at epoch 7
    Training LSTM for 10 epochs on cpu
    Epoch   1/10: Train=1.262625, Val=0.915135
    Epoch   5/10: Train=1.261065, Val=0.915395
    Early‑stopped at epoch 6
    Training LSTM for 10 epochs on cpu
    Epoch   1/10: Train=1.230929, Val=0.773627
    Epoch   5/10: Train=1.229839, Val=0.773783
    Epoch  10/10: Train=1.229291, Val=0.772826
    Training LSTM for 10 epochs on cpu
    Epoch   1/10: Train=1.895309, Val=0.842421
    Epoch   5/10: Train=1.889231, Val=0.842859
    Early‑stopped at epoch 6
    Training LSTM for 10 epochs on cpu
    Epoch   1/10: Train=1.265529, Val=0.914276
    Epoch   5/10: Train=1.262759, Val=0.914160
    Early‑stopped at epoch 9
    Training LSTM for 10 ep

In [19]:
run_portfolio_simulation_daily_rebalance()

Starting Daily Rebalance Portfolio Backtesting Simulation
Processing window size: 5
  Model: LSTM, Scheme: VW
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to original scale
[Inverse Transform] Window 5: predictions and actuals converted to ori

(   scheme model  window portfolio_type  annual_return  annual_vol    sharpe  \
 0      VW  LSTM       5      long_only       0.186154    0.225748  0.824609   
 1      VW  LSTM       5     short_only      -0.096422    0.202947 -0.475109   
 2      VW  LSTM       5     long_short       0.089732    0.237942  0.377118   
 3      EW  LSTM       5      long_only       0.152240    0.236384  0.644036   
 4      EW  LSTM       5     short_only      -0.143782    0.210357 -0.683514   
 5      EW  LSTM       5     long_short       0.008458    0.225746  0.037466   
 6      VW  LSTM      21      long_only       0.218543    0.217522  1.004692   
 7      VW  LSTM      21     short_only      -0.055203    0.171268 -0.322323   
 8      VW  LSTM      21     long_short       0.163339    0.205997  0.792919   
 9      EW  LSTM      21      long_only       0.231820    0.230541  1.005547   
 10     EW  LSTM      21     short_only      -0.027958    0.167504 -0.166907   
 11     EW  LSTM      21     long_short 

In [20]:
# ---------- Main function for 5-factor regression -----------
def run_factor_regression(port_ret, factors, use_excess=True):
    df = pd.concat([port_ret, factors], axis=1, join='inner').dropna()
    df.columns = ['ret'] + list(factors.columns)
    
    if use_excess:
        y = df['ret'].values
    else:
        y = df['ret'].values - df['rf'].values
    
    X = df[['mktrf','smb','hml','rmw','cma','umd']].values
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X)
    res = model.fit()
    alpha = res.params[0]          
    resid_std = res.resid.std(ddof=1)

    ir_daily = alpha / resid_std          
    ir_annual = ir_daily * np.sqrt(252)   

    y_hat = np.asarray(res.fittedvalues)  
    
    out = {
        'N_obs'            : len(y),
        'alpha_daily'      : alpha,
        'alpha_annual'     : alpha*252,      
        't_alpha'          : res.tvalues[0],
        'IR_daily'         : ir_daily,
        'IR_annual'        : ir_annual,
        'R2_zero'          : r2_zero(y, y_hat),
    }
    
    factor_names = ['MKT','SMB','HML','RMW','CMA','UMD']
    for i, fac in enumerate(factor_names, start=1):
        out[f'beta_{fac}'] = res.params[i]
        out[f't_{fac}']    = res.tvalues[i]
    
    return out

# ---------- Batch run (EW/VW, three portfolio types) ----------
def batch_factor_analysis(
    daily_df: pd.DataFrame,
    factors_path: str,
    scheme: str,
    tc_levels=(0, 5, 10, 20, 40),
    portfolio_types=('long_only','short_only','long_short'),
    model_filter=None,
    window_filter=None,
    gross_only=False,            
    out_dir='factor_IR_results',
):
    """
    Generate a CSV containing IR results.
    gross_only=True  → only tc=0; False → all tc_levels.
    """
    import os
    os.makedirs(out_dir, exist_ok=True)

    fac = (pd.read_csv(factors_path, parse_dates=['date'])
             .set_index('date')
             .sort_index())

    sub = daily_df[daily_df['scheme'] == scheme].copy()
    if model_filter is not None:
        sub = sub[sub['model'].isin(model_filter)]
    if window_filter is not None:
        sub = sub[sub['window'].isin(window_filter)]

    tc_iter = (0,) if gross_only else tc_levels
    results = []

    for (model, win, ptype), g in sub.groupby(['model','window','portfolio_type']):
        g = g.sort_values('date').set_index(pd.to_datetime(g['date']))

        for tc in tc_iter:
            col = 'return' if tc == 0 else f'tc{tc}_return'
            if col not in g.columns:
                continue  
            port_ret = g[col]
            stats = run_factor_regression(port_ret, fac, use_excess=True)
            stats.update({
                'scheme'        : scheme,
                'model'         : model,
                'window'        : win,
                'portfolio_type': ptype,
                'tc_bps'        : tc,
            })
            results.append(stats)

    df_out = pd.DataFrame(results)[[
        'scheme','model','window','portfolio_type','tc_bps','N_obs',
        'alpha_daily','alpha_annual','t_alpha',
        'IR_daily','IR_annual','R2_zero',
        'beta_MKT','t_MKT','beta_SMB','t_SMB',
        'beta_HML','t_HML','beta_RMW','t_RMW',
        'beta_CMA','t_CMA','beta_UMD','t_UMD'
    ]]

    tag = 'gross' if gross_only else 'net'
    fname = f'5_factor_analysis_{scheme}_{tag}.csv'
    df_out.to_csv(os.path.join(out_dir, fname), index=False)
    print(f'[Saved] {fname}')
    return df_out



def run_all_factor_tests(vw_csv="portfolio_daily_series_VW.csv",
                         ew_csv="portfolio_daily_series_EW.csv",
                         factor_csv="/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/5_Factors_Plus_Momentum.csv",
                         save_dir="results",
                         y_is_excess=True,
                         hac_lags=5,
                         save_txt=True):
    vw_df = pd.read_csv(vw_csv)
    ew_df = pd.read_csv(ew_csv)

    vw_gross = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=True)    
    vw_net   = batch_factor_analysis(
        vw_df, factor_csv, scheme='VW', gross_only=False)   

    ew_gross = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=True)
    ew_net   = batch_factor_analysis(
        ew_df, factor_csv, scheme='EW', gross_only=False)

    return vw_gross, vw_net, ew_gross, ew_net
    

vw_gross, vw_net, ew_gross, ew_net = run_all_factor_tests()

[Saved] 5_factor_analysis_VW_gross.csv 
[Saved] 5_factor_analysis_VW_net.csv 
[Saved] 5_factor_analysis_EW_gross.csv 
[Saved] 5_factor_analysis_EW_net.csv 


In [21]:
# === File Paths ===
rf_file = "/Users/june/Documents/University of Manchester/Data Science/ERP/Project code/1_Data_Preprocessing/CRSP_2016_2024_top50_with_exret.csv"
vw_file = "portfolio_daily_series_VW.csv"
ew_file = "portfolio_daily_series_EW.csv"

# === Load risk-free rate (rf) data ===

rf_df = pd.read_csv(rf_file, usecols=["date", "rf"])
rf_df["date"] = pd.to_datetime(rf_df["date"])
rf_dict = dict(zip(rf_df["date"], rf_df["rf"]))


def adjust_returns_with_rf_grouped(file_path, output_path):
    df = pd.read_csv(file_path)
    # Handle different date formats
    df["date"] = pd.to_datetime(df["date"], format='mixed', dayfirst=True)

    # Find all return columns (excluding cumulative columns)
    return_cols = [col for col in df.columns if "return" in col and "cumul" not in col]

    # Set portfolio_type order to avoid groupby sorting issues
    order = ["long_only", "short_only", "long_short"]
    df["portfolio_type"] = pd.Categorical(df["portfolio_type"], categories=order, ordered=True)

    df_list = []
    # Group by scheme/model/window/portfolio_type, add rf and recalculate cumulative
    for _, group in df.groupby(["scheme", "model", "window", "portfolio_type"], sort=False):
        group = group.sort_values("date").copy()
        for col in return_cols:
            group[col] = group.apply(lambda row: row[col] + rf_dict.get(row["date"], 0), axis=1)
            cum_col = col.replace("return", "cumulative")
            group[cum_col] = np.log1p(group[col]).cumsum()
        df_list.append(group)

    df_new = pd.concat(df_list).sort_values(["scheme", "model", "window", "portfolio_type", "date"])
    df_new.to_csv(output_path, index=False)
    print(f"Finished: {output_path}")

# Process VW and EW files
adjust_returns_with_rf_grouped(vw_file, "portfolio_daily_series_VW_with_rf.csv")
adjust_returns_with_rf_grouped(ew_file, "portfolio_daily_series_EW_with_rf.csv")


Finish: portfolio_daily_series_VW_with_rf.csv
Finish: portfolio_daily_series_EW_with_rf.csv


In [22]:
# ======== Download S&P500 (2016-2024) ========
sp500 = yf.download("^GSPC", start="2016-01-01", end="2024-12-31")
price_col = "Adj Close" if "Adj Close" in sp500.columns else "Close"
sp500["daily_return"] = sp500[price_col].pct_change().fillna(0)
sp500["cum_return"] = np.cumsum(np.log1p(sp500["daily_return"]))
sp500 = sp500[["cum_return"]]
sp500.index = pd.to_datetime(sp500.index)

# ======== Configuration ========
files = [
    ("VW", "portfolio_daily_series_VW_with_rf.csv"),
    ("EW", "portfolio_daily_series_EW_with_rf.csv")
]
tc_levels = [0, 5, 10, 20, 40]      # Transaction cost (bps)
windows = [5, 21, 252, 512]         # Window sizes
strategies = ["long_only", "short_only", "long_short"]

output_dir = "Baseline_Portfolio"
os.makedirs(output_dir, exist_ok=True)

# Economic crisis periods (for shading in plots)
crisis_periods = [
    (datetime(2018, 6, 1), datetime(2019, 1, 1), "US-China Trade War"),
    (datetime(2020, 2, 1), datetime(2020, 7, 1), "COVID-19"),
    (datetime(2022, 2, 1), datetime(2022, 6, 1), "Russia-Ukraine War"),
    (datetime(2023, 1, 1), datetime(2023, 4, 1), "US Bank Crisis"),
]

def plot_comparison_styled(df, scheme, tc, window):
    plt.figure(figsize=(15, 12))
    model_names = df["model"].unique()
    colors = plt.cm.tab10(np.linspace(0, 1, len(model_names)))

    offset_step = 0.02

    for i, strat in enumerate(strategies, 1):
        ax = plt.subplot(3, 1, i)

        plt.plot(sp500.index, sp500["cum_return"],
                 color="black", lw=2.5, label="S&P500 (Total Return)", zorder=10)

        for idx, model_name in enumerate(model_names):
            sub = df[(df["window"] == window) &
                     (df["portfolio_type"] == strat) &
                     (df["model"] == model_name)].sort_values("date")
            if sub.empty:
                continue

            if tc == 0:
                ret_col = "return"
            else:
                ret_col = f"tc{tc}_return"

            if ret_col not in sub.columns:
                continue

            log_cum = np.cumsum(np.log1p(sub[ret_col].values))

            y_shift = idx * offset_step
            plt.plot(sub["date"], log_cum + y_shift,
                     label=f"{model_name} ({strat.replace('_',' ').title()})",
                     lw=2, color=colors[idx], alpha=0.9)

        for start, end, label in crisis_periods:
            ax.axvspan(start, end, color='grey', alpha=0.3)
            ax.text(start + pd.Timedelta(days=10),
                    ax.get_ylim()[1]*0.92, label, fontsize=8, color='grey')
        ax.xaxis.set_major_locator(mdates.YearLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
        ax.set_ylabel("Cumulative log return (start = 0)")
        ax.set_title(f"{scheme} | Window={window} | Strategy={strat} | TC={tc} bps")
        ax.grid(alpha=0.3)
        plt.xticks(rotation=30)
        plt.legend(bbox_to_anchor=(1.04, 1), loc='upper left', fontsize=8)

    plt.tight_layout()
    fname = f"{scheme}_window{window}_TC{tc}_logreturn_offset.png"
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()

# ======== Main loop to generate all figures ========
for scheme, file_path in files:
    df = pd.read_csv(file_path)
    df["date"] = pd.to_datetime(df["date"])
    for tc in tc_levels:
        for window in windows:
            plot_comparison_styled(df, scheme, tc, window)

print(f"All figures have been generated and saved to: {output_dir}/")


[*********************100%***********************]  1 of 1 completed


All figures have been generated and saved to: Baseline_Portfolio/


In [23]:

# Load R²_zero from portfolio_metrics.csv
metrics_df = pd.read_csv("portfolio_metrics.csv")[["Model", "Window", "R²_zero"]]
metrics_df.rename(columns={"Model": "model", "Window": "window"}, inplace=True)

# Process VW/EW files
for fname in ["portfolio_results_daily_rebalance_VW.csv", "portfolio_results_daily_rebalance_EW.csv"]:
    df = pd.read_csv(fname)

    # Merge R²_zero by model and window
    df = df.merge(metrics_df, on=["model", "window"], how="left")

    rows = []
    for _, row in df.iterrows():
        r2 = float(row["R²_zero"]) if not pd.isna(row["R²_zero"]) else 0.0
        if row["portfolio_type"] == "long_only":
            d_sr, sr_star = delta_sharpe(r2, SR_MKT_EX)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = f"SPX_excess ({SR_MKT_EX:.2f})"
        else:
            d_sr, sr_star = delta_sharpe(r2, 0)
            row["ΔSharpe"]  = d_sr
            row["Sharpe*"]  = sr_star
            row["baseline"] = "cash (0)"
        rows.append(row)

    pd.DataFrame(rows).to_csv(fname, index=False)
    print(f"[Update] ΔSharpe has been written to {fname}")

[Update] ΔSharpe has been written to portfolio_results_daily_rebalance_VW.csv
[Update] ΔSharpe has been written to portfolio_results_daily_rebalance_EW.csv


In [1]:
import pandas as pd
import numpy as np
from math import sqrt

PRED_PATH = "predictions_daily.csv"
METRICS_PATH = "portfolio_metrics.csv"
TREAT_CONSTANT_DAY_AS_ZERO = False
MIN_DAYS_FOR_STATS = 1

def _day_ic(g):
    if g["y_pred"].nunique(dropna=True) <= 1 or g["y_true"].nunique(dropna=True) <= 1:
        return 0.0 if TREAT_CONSTANT_DAY_AS_ZERO else np.nan
    return g["y_pred"].corr(g["y_true"], method="spearman")

def rankic_stats(df_group):
    ics = (df_group.groupby("signal_date", observed=True).apply(_day_ic).dropna())
    n = int(ics.shape[0])
    if n < MIN_DAYS_FOR_STATS:
        return pd.Series({"RankIC_mean": np.nan, "RankIC_t": np.nan, "RankIC_pos%": np.nan, "N_days": n})
    mean_ic = float(ics.mean())
    std_ic  = float(ics.std(ddof=1))
    t_ic    = mean_ic / (std_ic / np.sqrt(n)) if std_ic > 0 else np.nan
    pos_pct = float((ics > 0).mean())
    return pd.Series({"RankIC_mean": mean_ic, "RankIC_t": t_ic, "RankIC_pos%": pos_pct, "N_days": n})

# Read data and calculate RankIC
pred = pd.read_csv(PRED_PATH)
pred["signal_date"] = pd.to_datetime(pred["signal_date"], errors="coerce")
pred = pred.dropna(subset=["signal_date", "y_true", "y_pred", "model", "window"])
pred["window"] = pd.to_numeric(pred["window"], errors="coerce").astype("Int64")

rankic_df = (pred.groupby(["model", "window"], dropna=False)
                .apply(rankic_stats)
                .reset_index()
                .rename(columns={"model":"Model","window":"Window"}))

# Merge: keep new RankIC columns, add _old suffix to original metrics columns
metrics = pd.read_csv(METRICS_PATH)
metrics["Window"] = pd.to_numeric(metrics["Window"], errors="coerce").astype("Int64")

merged = metrics.merge(rankic_df, on=["Model","Window"], how="left", suffixes=("_old",""))

# Drop old columns with _old suffix
to_drop = [c for c in merged.columns if c.endswith("_old")]
merged = merged.drop(columns=to_drop)

# Save and overwrite
merged.to_csv(METRICS_PATH, index=False)
print("[OK] Overwrote portfolio_metrics.csv with new RankIC")


/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_61715/1104344638.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ics = (df_group.groupby("signal_date", observed=True).apply(_day_ic).dropna())
/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_61715/1104344638.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ics = (df_group.groupby("signal_date", observed=True).apply(_day_ic).drop

[OK] Overwrote portfolio_metrics.csv with new RankIC )


/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_61715/1104344638.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ics = (df_group.groupby("signal_date", observed=True).apply(_day_ic).dropna())
/var/folders/wj/7nbyftys3g77fcp37r3t0_lm0000gn/T/ipykernel_61715/1104344638.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rankic_stats)
